## Polsky Center Compliance Analysis

The goal of this notebook is to determine if active licensing agreements have an associated compliance record. For those that do not have an associated compliance record, further analysis is conducted to link these inventions to individual divisions and departments 

### Load packages and datasets 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import scatter_matrix
import missingno as msno
import plotly.graph_objects as go
%matplotlib inline


import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', None)

In [2]:
# Agreement Dataset 
df_ag_full = pd.DataFrame(pd.read_csv("Agreement_cleaned.csv"))
print("Agreement Shape:", df_ag_full.shape)

# Technology Dataset 
df_tech = pd.DataFrame(pd.read_csv("Tech_cleaned.csv"))
print("Technology Shape:", df_tech.shape)

# Current Receivables Dataset 
df_fin = pd.DataFrame(pd.read_csv("Current_Receivables.csv"))
print("Receivables Shape:", df_fin.shape)

# Compliance Dataset 
df_comp = pd.DataFrame(pd.read_csv("ComplianceData.csv"))
print("Compliance Shape:", df_comp.shape)

Agreement Shape: (3843, 23)
Technology Shape: (3909, 86)
Receivables Shape: (578, 22)
Compliance Shape: (1528, 15)


### Include copies of Agreement dataset to be used for merge analysis

In [3]:
df_ag_check = df_ag_full[['Agreement_ID', 'Status']].copy()
df_ag = df_ag_full.copy()

### Agreements Dataset cleaning 
### We are *only* looking for Filter Status = Active in Agreements dataset. We are excluding the following statuses: Expired, Terminated, At Potential, Amended, In Negotiation, Negotiation Ended, URA: Assigned (In Process), URA: Submitted (Outstanding MOU). 

### Agreement_ID has 3,843 unique records, meaning that we have one unique agreement per record in the Agreements Dataset--as expected. Technologies are grouped together in one column. We will adjust the dataset so we have one unique Tech_ID and Agreement_ID per row 

In [4]:
df_ag.nunique()

Unnamed: 0               3843
Agreement_ID             3843
Parties                  2036
Agreement_Type             43
Manager                    13
Responsible_Manager         6
Title                    3559
Number_of_Parties          11
In_or_Out                   2
Reference_Number         1821
Status                     10
Status_Date              1235
Effective_Date           2094
Agreement_Term             27
Term_Date                1565
Sign_Date                1173
Field_Of_Use             1680
Equity                      2
Reimbursable_Expenses       2
Technologies             1032
Division_Department       191
Investigators             985
Last_Related_Update      3343
dtype: int64

### UPDATE: For Technologies with Agreement_ID's like 05-A-052.S9 and 05-A-052.S7, remove the S indicators at the end. These are sub-agreements under the same revenue-generating agreement 

In [5]:
# Break out individual Techology ID's 
df_ag = df_ag.assign(Tech_ID=df_ag['Technologies'].str.split(',')).explode('Tech_ID')

# Filter Stauts = Active in Agreements table. There are 630 active agreements in the agreements dataset
df_ag = df_ag.loc[df_ag['Status'] == 'Active'] 

### We see that there are no duplicate  Agreement_ID/Tech_ID combinations. Each row in the dataset is a unique  Agreement_ID/Tech_ID combination

In [6]:
duplicates = df_ag[df_ag.duplicated(subset=['Agreement_ID','Tech_ID'], keep=False)]
duplicates.shape

(0, 24)

### There are 630 unique active agreements, 644 unique technologies with an active agreement and 1,311 unique techology/agreement combinations. The many-to-many relationship between Agreements and Technologies accounts for the inflated techology/agreement combinations 

In [7]:
print("Count of unique techologies with an active licensing agreement:", df_ag['Tech_ID'].nunique())

print("Count of unique active licensing agreements:", df_ag['Agreement_ID'].nunique())

print("Count of unique technology/active license combinations:", df_ag.shape)

Count of unique techologies with an active licensing agreement: 644
Count of unique active licensing agreements: 630
Count of unique technology/active license combinations: (1311, 24)


### There are 117 agreements with associated receivables data. Of these 117 agreements with receivables data, 80 are connected to *active* agreements. 

In [8]:
df_fin.nunique()

Agreement ID                     117
Category                           3
Receivable Status                  1
Comments                          20
Party Name                       104
Attn                               4
Agreement Manager                  5
Agreement Responsible Manager      3
Invoice Number                   358
Party Balance Remaining          360
Party Amount Due                 357
Party Paid to Date                18
Total Revenue                    102
Total Reimbursable Expense       260
PO Number                        172
Invoice Date                     239
Party Last Paid Date              16
Due Date                         243
Payment Term                       1
Last Printed Date                  1
Type                               3
Archive Date                       1
dtype: int64

### Add 'Revenue_Generating' binary indicator column for Receivables dataset. If 'Total_Revenue' in Financials table is >0 then 1, if not then 0

In [9]:
# Select columns of interest from Receivables dataset and add 'Revenue_Generating' binary indicator column  
# If 'Total_Revenue' in Financials table is >0 then 1
df_fin = df_fin[['Agreement ID', 'Category', 'Total Revenue']].copy()

df_fin.loc[df_fin['Total Revenue'] > 0, 'Revenue_Generating'] = 1
df_fin.loc[df_fin['Total Revenue'] <= 0, 'Revenue_Generating'] = 0

#reanme Agreement_ID prior to merge 
df_fin = df_fin.rename(columns={'Agreement ID': 'Agreement_ID'})

df_fin.head()

,Agreement_ID,Category,Total Revenue,Revenue_Generating
0,19-A-0121,Revenue,300000.00,1.0
1,21-A-159,Reimbursement,0.00,0.0
2,05-A-042,Reimbursement,0.00,0.0
3,16-A-184,Both,91257.49,1.0
4,21-A-159,Reimbursement,0.00,0.0


In [10]:
df_fin.shape

(578, 4)

### Agreements in the Receivables dataset can have some records with revenue and some without revenue. See Agreement_ID = 05-A-042 below

In [11]:
duplicates_fin = df_fin[df_fin.duplicated(subset=['Agreement_ID'], keep=False)]
duplicates_fin[duplicates_fin['Agreement_ID'].str.match('05-A-042')]

,Agreement_ID,Category,Total Revenue,Revenue_Generating
2,05-A-042,Reimbursement,0.00,0.0
15,05-A-042,Reimbursement,0.00,0.0
33,05-A-042,Revenue,43115.13,1.0
36,05-A-042,Revenue,35651.20,1.0
48,05-A-042,Revenue,28544.75,1.0
50,05-A-042,Revenue,27217.65,1.0
61,05-A-042,Revenue,23757.89,1.0
65,05-A-042,Revenue,22549.40,1.0
66,05-A-042,Revenue,22145.83,1.0
67,05-A-042,Revenue,21566.99,1.0


### Because we are interested in unique agreements, we will group by Agreement_ID and only keep the max value from the Revenue_Generating column. This will identify agreements with *any* associated revenue as being revenue generating, even if other rows with that Agreement_ID do not have associated revenue

In [12]:
df_fin = df_fin.loc[df_fin.groupby(['Agreement_ID'])['Revenue_Generating'].idxmax()].reset_index(drop=True)

df_fin.shape

(117, 4)

In [13]:
df_fin['Agreement_ID'].nunique()

117

### Now that we are only keeping unque Agreements from the Receivables dataset, we will merge with the agreements dataset to identify revenue-generating technologiy/agreement combinations

In [14]:
merge_revenue = df_ag.merge(df_fin.drop_duplicates(), on=['Agreement_ID'], 
                   how='outer', indicator=True)

### After merging the Receivables data with Active Agreements, we can see that we have the same number of unique technologies, but we have 37 Agreement_ID records from the Receivables dataset that are not in the Agreement dataset

In [15]:
print("Count of unique techologies with an active licensing agreement in the merged dataset:", merge_revenue['Tech_ID'].nunique())

print("Count of unique licensing agreements in the merged dataset:", merge_revenue['Agreement_ID'].nunique())

print("Count of unique technology/license combinations in the merged dataset:", merge_revenue.shape)

Count of unique techologies with an active licensing agreement in the merged dataset: 644
Count of unique licensing agreements in the merged dataset: 667
Count of unique technology/license combinations in the merged dataset: (1348, 28)


### We can see that we have 80 *Active* Agreements within the Agreement dataset that have associated revenue data.

In [16]:
receivable_data = merge_revenue.loc[merge_revenue['_merge'] == 'both']

print("Count of unique techologies with an active licensing agreement and revenue data:", receivable_data['Tech_ID'].nunique())

print("Count of unique licensing agreements with revenue data:", receivable_data['Agreement_ID'].nunique())

print("Count of unique technology/license combinations with revenue data:", receivable_data.shape)

Count of unique techologies with an active licensing agreement and revenue data: 179
Count of unique licensing agreements with revenue data: 80
Count of unique technology/license combinations with revenue data: (180, 28)


### This is a list of the records with receivable data that are not active agreements

In [17]:
missing_agreement = merge_revenue.loc[merge_revenue['_merge'] == 'right_only']

missing_agreement

,Unnamed: 0,Agreement_ID,Parties,Agreement_Type,Manager,Responsible_Manager,Title,Number_of_Parties,In_or_Out,Reference_Number,...,Reimbursable_Expenses,Technologies,Division_Department,Investigators,Last_Related_Update,Tech_ID,Category,Total Revenue,Revenue_Generating,_merge
1311,NaN,01-A-042,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Revenue,229.00,1.0,right_only
1312,NaN,05-A-041,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Reimbursement,0.00,0.0,right_only
1313,NaN,05-A-042,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Revenue,43115.13,1.0,right_only
1314,NaN,07-A-049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Revenue,50000.00,1.0,right_only
1315,NaN,07-A-086,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Revenue,77612.15,1.0,right_only
1316,NaN,08-A-025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Reimbursement,0.00,0.0,right_only
1317,NaN,08-A-046,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Revenue,57759.94,1.0,right_only
1318,NaN,10-A-089,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Reimbursement,0.00,0.0,right_only
1319,NaN,11-A-014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Revenue,10000.00,1.0,right_only
1320,NaN,11-A-068,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Reimbursement,0.00,0.0,right_only


### Checking the cases where we have receivables data but nothing in the Agreement dataset, 33 out of the 37 cases are agreements that are listed as not being active in the Agreement dataset.

### The additional missing records: 21-A-218, 21-A-219, 21-A-220, do not appear in the full Agreement dataset

In [18]:
missing_agreement_check = df_ag_check.merge(missing_agreement.drop_duplicates(), on=['Agreement_ID'], 
                   how='inner')

missing_agreement_check

,Agreement_ID,Status_x,Unnamed: 0,Parties,Agreement_Type,Manager,Responsible_Manager,Title,Number_of_Parties,In_or_Out,...,Reimbursable_Expenses,Technologies,Division_Department,Investigators,Last_Related_Update,Tech_ID,Category,Total Revenue,Revenue_Generating,_merge
0,01-A-042,Expired,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Revenue,229.00,1.0,right_only
1,05-A-041,Terminated,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Reimbursement,0.00,0.0,right_only
2,05-A-042,Amended,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Revenue,43115.13,1.0,right_only
3,07-A-049,Expired,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Revenue,50000.00,1.0,right_only
4,07-A-086,Terminated,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Revenue,77612.15,1.0,right_only
5,08-A-025,Amended,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Reimbursement,0.00,0.0,right_only
6,08-A-046,Amended,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Revenue,57759.94,1.0,right_only
7,10-A-089,Amended,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Reimbursement,0.00,0.0,right_only
8,11-A-014,Amended,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Revenue,10000.00,1.0,right_only
9,11-A-068,Amended,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Reimbursement,0.00,0.0,right_only


### Because the 37 cases with receivable data missing from the Agreement dataset appear to be expired or not relevant, we will remove from the merged dataset

In [19]:
merge_revenue = merge_revenue[~merge_revenue._merge.str.contains("right_only")]

merge_revenue.shape

(1311, 28)

### We will make the assumption that, if a technology/agreement combination does not have corresponding receivables data, it is non-revenue generating. Therefore, if Revenue_Generating = NA, we will convert to 0 (non-revenue generating) 

In [20]:
merge_revenue['Revenue_Generating'].fillna(0, inplace=True)

### Now that we have merged the Agreements dataset with the Revenue Dataset, we will combine this with the Technology table via Tech_ID 

### There are 644 unique technologies with an active licensing agreement in the Agreement Dataset and 390 unique technologies with a 'licensed' status in the Technology dataset. 

### Kyle is reviewing the large discrepancy between licensed technologies and active agreements 

In [21]:
# Generate lists for the will never be licensed, could be licensed, igone, and licensed statuses
licensed = ('Non-Exclusively Licensed', 'Exclusively Licensed', 'Optioned')
could_license = ('Seeking Licensees', 'Post Election Hold', 'IP Authorized', 'Pending Title Election Decision')
never_licensed = ('Closed/Inactive', 'Waived Rights to Inventor', 'Awaiting Expiration')
ignore = ('Licenses at Potential', 'IIA - Other Party Leads', 'Jointly Owned - UoC Leads', 'Combined with other Tech', 'Jointly Owned - Other Party Leads', 'Awaiting Info from Inventors', 'Negotiating License')

df_tech.loc[df_tech['Status'].isin(licensed), 'License_Status'] = 1
df_tech.loc[df_tech['Status'].isin(could_license), 'License_Status'] = 2
df_tech.loc[df_tech['Status'].isin(never_licensed), 'License_Status'] = 3
df_tech.loc[df_tech['Status'].isin(ignore), 'License_Status'] = 4

# Keep only technologies with a status of licensed and keep only columns of interest 
df_tech = df_tech.loc[df_tech['License_Status'] == 1]
df_tech = df_tech[['Tech_ID', 'Invention_ID', 'Lead_Inventor', 'Owners']].copy()

df_tech.shape

(390, 4)

### We can see that, after merging the Technology dataset with the Agreement dataset, we have the same number of unique agreements, but 280 more unique technologies and technology/agreement combinations 

In [22]:
merge_revenue = merge_revenue.drop('_merge', 1)

merge_tech = merge_revenue.merge(df_tech.drop_duplicates(), on=['Tech_ID'], 
                   how='outer', indicator=True)

In [23]:
print("Count of unique techologies with an active licensing agreement in the merged dataset:", merge_tech['Tech_ID'].nunique())

print("Count of unique licensing agreements in the merged dataset:", merge_tech['Agreement_ID'].nunique())

print("Count of unique technology/license combinations in the merged dataset:", merge_tech.shape)

Count of unique techologies with an active licensing agreement in the merged dataset: 924
Count of unique licensing agreements in the merged dataset: 630
Count of unique technology/license combinations in the merged dataset: (1591, 31)


### Of the 924 unique Tech_ID's in the merged dataset, 110 are found in both datasets, 534 are only found in the Agreements dataset, and 280 are only found in the Technology dataset. The 280 technologies only found in the Technology dataset account for the additional technology/agreement combinations 

In [24]:
tech_data_both = merge_tech.loc[merge_tech['_merge'] == 'both']

tech_data_agreement = merge_tech.loc[merge_tech['_merge'] == 'left_only']

tech_data_tech = merge_tech.loc[merge_tech['_merge'] == 'right_only']

print("Technologies found in both the agreement and tech datasets:", tech_data_both['Tech_ID'].nunique())

print("Technologies found only in the agreements dataset:", tech_data_agreement['Tech_ID'].nunique())

print("Technologies found only in the technology dataset:", tech_data_tech['Tech_ID'].nunique())

Technologies found in both the agreement and tech datasets: 110
Technologies found only in the agreements dataset: 534
Technologies found only in the technology dataset: 280


### We will need to do additional analysis on the 280 records from the Technology dataset that are not in the Agreement dataset

In [25]:
# Select only columns from Technology dataset
tech_data_tech = tech_data_tech[['Tech_ID', 'Invention_ID', 'Lead_Inventor']].copy()
tech_data_tech.head()

,Tech_ID,Invention_ID,Lead_Inventor
1311,00-T-009,00-D-009,Jeffrey Bluestone
1312,00-T-010,00-D-010,Samuel Armato
1313,00-T-046,00-D-046,David Grier
1314,01-T-008,01-D-008,Samuel Armato
1315,01-T-016,01-D-016,Kunio Doi


In [26]:
# Add Tech_ID to full agreements dataset (this includes active and inactive agreements)
df_ag_full = df_ag_full.assign(Tech_ID=df_ag_full['Technologies'].str.split(',')).explode('Tech_ID')

In [27]:
merge_tech_full = tech_data_tech.merge(df_ag_full.drop_duplicates(), on=['Tech_ID'], 
                   how='left')

### The 280 records listed as 'licensed' in the Technology dataset are not found in the Agreement dataset because they are inactive agreements. This includes Tech_ID's with an Agreement Status of  Amended, At Potential, Expired, In Negotiation, Negotiation Ended, Terminated, and Blank values

In [28]:
print("Unique technologies from Technology Dataset with an inactive agreement:", merge_tech_full['Tech_ID'].nunique())

Unique technologies from Technology Dataset with an inactive agreement: 280


### Because the 280 records from the Technology Dataset not found in the Merged Dataset have a status of inactive or blank, we will remove them from the Merged Dataset 

In [29]:
final_df = merge_tech.loc[merge_tech['_merge'] != 'right_only']

In [30]:
print("Unique Technologies in the Merged Dataset:", final_df['Tech_ID'].nunique())

print("Unique Technologies in the Merged Dataset:", final_df['Agreement_ID'].nunique())

print("Shape of the Merged dataset:", final_df.shape)

Unique Technologies in the Merged Dataset: 644
Unique Technologies in the Merged Dataset: 630
Shape of the Merged dataset: (1311, 31)


### Export Active Revenue-Generating/Non-Revenue-Generating dataset for review with the Polsky Center 

In [31]:
#final_df.to_csv(r"C:\Users\Nick's Laptop\Desktop\Capstone\gitStaging\final_df.csv", sep=',')

### Select columns of interest from Compliance dataset and prep for merging. Any technologies within the Compliance Dataset are considered *non-compliant*

In [32]:
# select columns of interest from compliance dataset. Where can we pull in owner?
df_comp = df_comp[['Tech ID', 'IR Code', 'Subject']].copy()

#reanme Agreement_ID prior to merge 
df_comp = df_comp.rename(columns={'Tech ID': 'Tech_ID'})

# keep only unique records from the compliance dataset 
df_comp = df_comp.drop_duplicates()

# review unique values within columns and shape 
print (df_comp.shape)

print(df_comp.nunique())

(1283, 3)
Tech_ID    467
IR Code     29
Subject     56
dtype: int64


In [33]:
final_df = final_df.drop('_merge', 1)

df_final = final_df.merge(df_comp.drop_duplicates(), on=['Tech_ID'], 
                   how='left', indicator=True)

### After completing a left join of the Compliance dataset onto the Active Revenue/Non-Revenue dataset, we have the same number of unique Technologies and Agreements, but more total records due to the numerous IR Codes and Subjects associated with Agreement/Technology combinations

In [34]:
print("Unique Technologies in the Merged Dataset:", df_final['Tech_ID'].nunique())

print("Unique Technologies in the Merged Dataset:", df_final['Agreement_ID'].nunique())

print("Shape of the Merged dataset:", df_final.shape)

Unique Technologies in the Merged Dataset: 644
Unique Technologies in the Merged Dataset: 630
Shape of the Merged dataset: (1871, 33)


In [35]:
df_final.loc[df_final['_merge'] == 'left_only', 'Non_Compliant'] = 0
df_final.loc[df_final['_merge'] == 'both', 'Non_Compliant'] = 1

df_final['Non_Compliant'].value_counts()

0.0    1136
1.0     735
Name: Non_Compliant, dtype: int64

### Duplicate values stem from a number of different IR Codes and Subjects within the Compliance Dataset. 

In [36]:
duplicates = df_final[df_final.duplicated(subset=['Agreement_ID','Tech_ID'], keep=False)]
duplicates.shape

(707, 34)

### Separate Compliant and Non-Compliant Records

In [37]:
non_compliant_df = df_final.loc[df_final['Non_Compliant'] == 1]

compliant_df = df_final.loc[df_final['Non_Compliant'] == 0]

### Get a count of IR Codes, Subjects, Lead Inventors, and Owners Assocaited with Non-Compliant records 

In [38]:
non_compliant_df['IR Code'].value_counts()

200        106
260        106
280         89
233         88
240         72
220         71
130         37
100         27
221         23
TM  233     21
TM  200     21
TM  220     18
300         15
234         12
TM  221     11
160         10
310          3
223          2
TM  130      2
TM  240      1
Name: IR Code, dtype: int64

In [39]:
non_compliant_df['Subject'].value_counts()

An initial non-provisional patent application must be timely filed. (e.g. 37 CFR 401.14(c)(3)).                                                                                                                                                                        126
A Government Support Clause is missing for a non-provisional patent filing in this record.                                                                                                                                                                              84
A Government Support Clause is not accepted. See the “Government Support Clause Reject Comment” in the patent report screen. 37 CFR 401.14(f)(4).                                                                                                                       71
The Provisional Filing is due to expire.  Either a non-provisional or a PCT patent application must be timely filed.  At least sixty days (60) prior Notice must be given to the Government if the repo

In [40]:
non_compliant_df['Lead_Inventor'].value_counts()

Bryan Dickinson       90
Raymond Moellering    35
Imre Noth             24
Chuan He              23
Susan Lindquist       20
Olaf Schneewind       16
Xiaoyang Wu           12
Thomas Gajewski       12
Yang-Xin Fu           11
John Alverdy           8
Maciej Lesniak         7
Shohei Koide           7
Hatim Hassan           7
Janet Beissinger       6
Wenbin Lin             6
Brian Popko            5
David Grdina           5
Tong-Chuan He          5
Elizabeth McNally      4
Joe Garcia             3
Richard Kraig          2
Michael Thirman        2
Marsha Rosner          1
Maryellen Giger        1
Name: Lead_Inventor, dtype: int64

In [41]:
non_compliant_df['Owners'].value_counts()

University of Chicago                                 202
Rush University Medical Center                          6
Argonne National Laboratory, University of Chicago      1
Name: Owners, dtype: int64

### Keep only unique Agreement/Technology combinations and check the number of revenue generating vs. non-revenue generating records 

In [42]:
non_compliant_dedup_df=non_compliant_df.drop_duplicates(['Tech_ID','Agreement_ID'])

print("Unique Technologies in the Non_Compliant Dataset:", non_compliant_dedup_df['Tech_ID'].nunique())

print("Unique Agreements in the Non_Compliant Dataset:", non_compliant_dedup_df['Agreement_ID'].nunique())

print("Shape of the Non_Compliant dataset:", non_compliant_dedup_df.shape)

Unique Technologies in the Non_Compliant Dataset: 86
Unique Agreements in the Non_Compliant Dataset: 175
Shape of the Non_Compliant dataset: (175, 34)


### Only 13 Non-Compliant Technology/Agreement combinations are Revenue Generating 

In [43]:
non_compliant_dedup_df['Revenue_Generating'].value_counts()

0.0    162
1.0     13
Name: Revenue_Generating, dtype: int64

### Extract Non-Compliant records and send to Polsky Center for Analysis

In [45]:
#non_compliant_dedup_df.to_csv(r"C:\Users\Nick's Laptop\Desktop\Capstone\gitStaging\noncompliant_technologies.csv", sep=',')